# 자율 주행 경로 생성 및 PID 제어

* 성명 : 

In [2]:
import pygame
import numpy as np
import math
import sys

pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('지도 기반 경로 및 장애물 입력 + 자율주행')
clock = pygame.time.Clock()

WHITE = (255, 255, 255)
RED   = (255,  50,  50)
BLUE  = ( 50,  50, 255)
BLACK = (  0,   0,   0)

# 지도 및 차량 이미지 로드
background = pygame.image.load('./input/map for MWU_m.png').convert()
background = pygame.transform.scale(background, (WIDTH, HEIGHT))

car_img_raw = pygame.image.load('./input/car_r_1280_640.png').convert_alpha()
car_img = pygame.transform.scale(car_img_raw, (64, 32))

# 좌표 변환 함수
def world_to_screen(x, y):
    return int(x * 15), int(HEIGHT / 2 - y * 15)

def screen_to_world(sx, sy):
    return sx / 15, (HEIGHT / 2 - sy) / 15

# 차량 클래스
class Vehicle:
    def __init__(self, x=0, y=0, yaw=0, v=0):
        self.x = x
        self.y = y
        self.v = v
        self.yaw = yaw

    def update(self, a, delta, dt=0.1):
        L = 2.5
        self.x += self.v * math.cos(self.yaw) * dt
        self.y += self.v * math.sin(self.yaw) * dt
        self.yaw += self.v / L * math.tan(delta) * dt
        self.v += a * dt

# PID 제어기
class PID:
    def __init__(self, Kp, Ki, Kd):
        self.Kp, self.Ki, self.Kd = Kp, Ki, Kd
        self.integral = 0
        self.prev_error = 0

    def control(self, target, current, dt):
        error = target - current
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.prev_error = error
        return output

# 장애물 회피 함수 정의
def obstacle_avoidance(vehicle, obstacles, threshold=4.0, influence=1.0):
    steer_adjust = 0
    for ox, oy in obstacles:
        dx = ox - vehicle.x
        dy = oy - vehicle.y
        dist = math.hypot(dx, dy)
        if dist < threshold and dx > 0:  # 장애물이 앞에 있을 때만 회피
            direction = -np.sign(dy)     # 장애물이 위에 있으면 위로 회피
            steer_adjust += direction * math.atan2((OBSTACLE_RADIUS-np.abs(dy)),dist)
    return steer_adjust

# -----------------------------
# 1단계: 경로 + 장애물 입력 모드
# -----------------------------
path_screen_coords = []
obstacle_screen_coords = []

selecting = True
while selecting:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.MOUSEBUTTONDOWN:
            mods = pygame.key.get_mods()
            if event.button == 1:
                if mods & pygame.KMOD_SHIFT:
                    obstacle_screen_coords.append(event.pos)
                else:
                    path_screen_coords.append(event.pos)

            elif event.button == 3:
                selecting = False  # 입력 종료

    screen.blit(background, (0, 0))

    # 경로 시각화
    for point in path_screen_coords:
        pygame.draw.circle(screen, RED, point, 5)
    if len(path_screen_coords) > 1:
        pygame.draw.lines(screen, RED, False, path_screen_coords, 2)

    # 장애물 시각화
    for point in obstacle_screen_coords:
        pygame.draw.circle(screen, BLACK, point, 8)

    pygame.display.flip()
    clock.tick(60)

# 입력 좌표를 world 좌표로 변환
path_x, path_y = [], []
for sx, sy in path_screen_coords:
    x, y = screen_to_world(sx, sy)
    path_x.append(x)
    path_y.append(y)

obstacles = []
for sx, sy in obstacle_screen_coords:
    x, y = screen_to_world(sx, sy)
    obstacles.append((x, y))

# -----------------------------
# 2단계: 주행 시뮬레이션 시작
# -----------------------------
vehicle = Vehicle(x=path_x[0], y=path_y[0])
speed_pid = PID(1.0, 0.1, 0.05)
steering_pid = PID(0.1, 1.0, 0.2)
target_speed = 10.0
trace = []

running = True
while running:
    dt = 0.01
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    dx = np.array(path_x) - vehicle.x
    dy = np.array(path_y) - vehicle.y
    dist = np.hypot(dx, dy)
    target_idx = np.argmin(dist)
    if target_idx >= len(path_x) - 1:
        target_idx = len(path_x) - 1

    target_y = path_y[target_idx]
    steer = steering_pid.control(target_y, vehicle.y, dt)
    steer += obstacle_avoidance(vehicle, obstacles)

    accel = speed_pid.control(target_speed, vehicle.v, dt)
    vehicle.update(a=accel, delta=steer, dt=dt)
    trace.append((vehicle.x, vehicle.y))

    # 지도 및 그래픽
    screen.blit(background, (0, 0))

    # 경로
    for sx, sy in path_screen_coords:
        pygame.draw.circle(screen, RED, (sx, sy), 3)

    # 이동 흔적
    for tx, ty in trace:
        sx, sy = world_to_screen(tx, ty)
        if sx >= 800:  # if exceed limit of width
            running = False
        pygame.draw.circle(screen, BLUE, (sx, sy), 2)

    # 장애물
    for ox, oy in obstacles:
        sx, sy = world_to_screen(ox, oy)
        pygame.draw.circle(screen, BLACK, (sx, sy), 10)

    # 차량 표시
    car_sx, car_sy = world_to_screen(vehicle.x, vehicle.y)
    angle_deg = math.degrees(vehicle.yaw)
    rotated_car = pygame.transform.rotate(car_img, angle_deg)
    rect = rotated_car.get_rect(center=(car_sx, car_sy))
    screen.blit(rotated_car, rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()